### Making chinook_dw table to pull json/csv files from

In [177]:
import pandas as pd
from sqlalchemy import create_engine

In [178]:
host_name = "ds2002-project2-chinook.mysql.database.azure.com"
host_ip = "127.0.0.1"
port = "3306"

user_id = "sqladmin"
pwd = "Scout1210"


src_dbname = "chinook"
dst_dbname = "chinook_dw"


In [179]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [180]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
sqlEngine.execute(f"USE {dst_dbname};")

### Make customer and employee dimension

In [181]:
sql_customers = "SELECT * FROM chinook.Customer;"

try:
    df_customers = get_dataframe(user_id, pwd, host_name, src_dbname, sql_customers)
except:
    print ("Error: unable to fetch data")
    
df_customers.head(2)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5


In [182]:
sql_employees = "SELECT * FROM chinook.employee;"

try:
    df_employees = get_dataframe(user_id, pwd, host_name, src_dbname, sql_employees)
except:
    print ("Error: unable to fetch data")

df_employees.head(2)

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18,2002-08-14,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08,2002-05-01,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com


In [183]:
#mysql server does not recognize the character in name here "Stanisław" so I just changed it for an easy fix...
df_customers['FirstName'].iloc[48] = "Stanislaw"

#drop Email becuase not needed and Fax because high number of nulls, also others since repeated in invoice table
drop_cols = ['Email','Fax', 'Address', 'City','State','Country','PostalCode']
df_customers.drop(drop_cols, axis=1, inplace=True)

#Rename Id, to Key. Also rename SupportRepId to employee_key to match future tables
df_customers.rename(columns={"CustomerId":"customer_key", "SupportRepId": "employee_key"}, inplace=True)

df_customers.head(2)

/var/folders/x_/qkfj00656_qg6nsj5fclf5sc0000gn/T/ipykernel_94443/861759716.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_customers['FirstName'].iloc[48] = "Stanislaw"


,customer_key,FirstName,LastName,Company,Phone,employee_key
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,+55 (12) 3923-5555,3
1,2,Leonie,Köhler,None,+49 0711 2842222,5


In [184]:
#Email, Fax, Birth date, and hire date not needed, Drop them
drop_cols = ['Email','Fax', 'BirthDate', 'HireDate']
df_employees.drop(drop_cols, axis=1, inplace=True)

#Rename Id to Key
df_employees.rename(columns={"EmployeeId":"employee_key"}, inplace=True)

df_employees.head(2)

,employee_key,LastName,FirstName,Title,ReportsTo,Address,City,State,Country,PostalCode,Phone
0,1,Adams,Andrew,General Manager,NaN,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482
1,2,Edwards,Nancy,Sales Manager,1.0,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443


In [185]:
db_operation = "insert"

tables = [('dim_customer', df_customers, 'customer_key'),
          ('dim_employee', df_employees, 'employee_key')]

In [186]:
for table_name, dataframe, primary_key in tables:
    try:
        set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)
    except:
        print ("Error: unable to send data: " + table_name)

### 2.0. Create & Populate the Fact Table

In [187]:
invoices_sql = "SELECT * FROM chinook.Invoice;"

try:
    df_invoices = get_dataframe(user_id, pwd, host_name, src_dbname, invoices_sql)
except:
    print ("Error: unable to fetch data")

#no need to rename anything yet, already unique id name (InvoiceId)
df_invoices.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11,69 Salem Street,Boston,MA,USA,2113,13.86


In [188]:
invoice_details_sql = "SELECT * FROM chinook.InvoiceLine;"

try:
    df_invoice_details = get_dataframe(user_id, pwd, host_name, src_dbname, invoice_details_sql)
except:
    print ("Error: unable to fetch data")

#no need to rename yet, already unique id name (InvoiceLineId)
df_invoice_details.head()

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1


In [189]:
#merge with customer table in order for the fact table to have employee key (in customer table)
#ranem keys to match in order to merge 
df_invoices.rename(columns={"CustomerId":"customer_key"}, inplace=True)
#merge on customer key
df_invoices = pd.merge(df_invoices, df_customers, on = "customer_key", how = 'inner')
#drop everything, but employee_key
df_invoices.drop(["FirstName", 'LastName', "Company", 'Phone'], axis=1, inplace=True)


In [190]:
df_fact_invoices = pd.merge(df_invoices, df_invoice_details, on='InvoiceId', how='right')
df_fact_invoices.head()

,InvoiceId,customer_key,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,employee_key,InvoiceLineId,TrackId,UnitPrice,Quantity
0,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,5,1,2,0.99,1
1,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,5,2,4,0.99,1
2,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,4,3,6,0.99,1
3,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,4,4,8,0.99,1
4,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,4,5,10,0.99,1


In [191]:
df_fact_invoices.rename(columns={"TrackId":"product_key"}, inplace=True)

#drop unnecessary columns, ex: invoicelineid since no need for foreign reference since in same dataframe now
df_fact_invoices.drop(['InvoiceLineId', "InvoiceId", "Total"], axis=1, inplace=True)
df_fact_invoices.head()

,customer_key,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,employee_key,product_key,UnitPrice,Quantity
0,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,5,2,0.99,1
1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,5,4,0.99,1
2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,4,6,0.99,1
3,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,4,8,0.99,1
4,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,4,10,0.99,1


In [192]:
ordered_columns = ['customer_key','employee_key','product_key','InvoiceDate','BillingAddress','BillingCity',
                   'BillingState','BillingCountry','BillingPostalCode','UnitPrice',
                   'Quantity']
df_fact_invoices = df_fact_invoices[ordered_columns]

In [193]:
df_fact_invoices.insert(0, "fact_invoice_key", range(1, df_fact_invoices.shape[0]+1))
df_fact_invoices.head(5)

,fact_invoice_key,customer_key,employee_key,product_key,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,UnitPrice,Quantity
0,1,2,5,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,0.99,1
1,2,2,5,4,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,0.99,1
2,3,4,4,6,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,0.99,1
3,4,4,4,8,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,0.99,1
4,5,4,4,10,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,0.99,1


In [194]:
table_name = "fact_invoice"
primary_key = "fact_invoice_key"
db_operation = "insert"

try:
        set_dataframe(user_id, pwd, host_name, dst_dbname, df_fact_invoices, table_name, primary_key, db_operation)
except:
        print ("Error: unable to send data")

### Make product dimension

In [195]:
sql_album = "SELECT * FROM chinook.album;"

try:
    df_album = get_dataframe(user_id, pwd, host_name, src_dbname, sql_album)
except:
    print ("Error: unable to fetch data")
    
df_album.head(2)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2


In [196]:
sql_artist = "SELECT * FROM chinook.artist;"

try:
    df_artist = get_dataframe(user_id, pwd, host_name, src_dbname, sql_artist)
except:
    print ("Error: unable to fetch data")
    
df_artist.head(2)

,ArtistId,Name
0,1,AC/DC
1,2,Accept


In [197]:
sql_media = "SELECT * FROM chinook.mediatype;"

try:
    df_media = get_dataframe(user_id, pwd, host_name, src_dbname, sql_media)
except:
    print ("Error: unable to fetch data")
    
df_media.head(2)

,MediaTypeId,Name
0,1,MPEG audio file
1,2,Protected AAC audio file


In [198]:
sql_genre = "SELECT * FROM chinook.genre;"

try:
    df_genre = get_dataframe(user_id, pwd, host_name, src_dbname, sql_genre)
except:
    print ("Error: unable to fetch data")
    
df_genre.head(2)

,GenreId,Name
0,1,Rock
1,2,Jazz


In [199]:
sql_track = "SELECT * FROM chinook.track;"

try:
    df_track = get_dataframe(user_id, pwd, host_name, src_dbname, sql_track)
except:
    print ("Error: unable to fetch data")
    
df_track.head(2)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99


In [200]:
#artists can have many albums, so right outer join on ArtistId
df_album_artist = pd.merge(df_artist, df_album, on='ArtistId', how='right')
#Drop artistid since no reference to it anymore, all in one dataframe.
df_album_artist.drop(['ArtistId'], axis=1, inplace=True)
#rename ids to keys... make name and title more explicit 
df_album_artist.rename(columns={"AlbumId": "album_key", "Name":"artist","Title":'album'}, inplace=True)
df_album_artist.head()

,artist,album_key,album
0,AC/DC,1,For Those About To Rock We Salute You
1,Accept,2,Balls to the Wall
2,Accept,3,Restless and Wild
3,AC/DC,4,Let There Be Rock
4,Aerosmith,5,Big Ones


In [201]:
#rename ids to keys... make name more explicit 
df_genre.rename(columns={"GenreId": "genre_key", "Name":"genre"}, inplace=True)
df_genre.head()

,genre_key,genre
0,1,Rock
1,2,Jazz
2,3,Metal
3,4,Alternative & Punk
4,5,Rock And Roll


In [202]:
#rename ids to keys... make name more explicit 
df_media.rename(columns={"MediaTypeId": "media_key", "Name":"media"}, inplace=True)
df_media.head()

,media_key,media
0,1,MPEG audio file
1,2,Protected AAC audio file
2,3,Protected MPEG-4 video file
3,4,Purchased AAC audio file
4,5,AAC audio file


In [203]:
# rename columns, change ids to keys again so no confusion with relationship between tables
df_track.rename(columns={"TrackId":"product_key", "AlbumId": "album_key",'MediaTypeId':"media_key",'GenreId':'genre_key'}, inplace=True)
df_track.head(2)

,product_key,Name,album_key,media_key,genre_key,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99


In [204]:
#do left join since want to retain all tracks even if album_key is null, which it won't be but just to be safe
#join into new dataframe 
df_track_details = pd.merge(df_track, df_album_artist, on='album_key', how='left')
df_track_details.head()

,product_key,Name,album_key,media_key,genre_key,Composer,Milliseconds,Bytes,UnitPrice,artist,album
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,AC/DC,For Those About To Rock We Salute You
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99,Accept,Balls to the Wall
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99,Accept,Restless and Wild
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99,Accept,Restless and Wild
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99,Accept,Restless and Wild


In [205]:
#pass new dataframe to be joined with genre
#same as above, want to retain all tracks even if genre is null. Use left join to be safe
#dont run more than once without rerunning the merge above
df_track_details = pd.merge(df_track_details, df_genre, on='genre_key', how='left')
df_track_details.head()

,product_key,Name,album_key,media_key,genre_key,Composer,Milliseconds,Bytes,UnitPrice,artist,album,genre
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,AC/DC,For Those About To Rock We Salute You,Rock
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99,Accept,Balls to the Wall,Rock
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99,Accept,Restless and Wild,Rock
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99,Accept,Restless and Wild,Rock
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99,Accept,Restless and Wild,Rock


In [206]:
#pass same dataframe to be joined with media, same as above, left outer join
#dont run more than once without rerunning the merges above
df_track_details = pd.merge(df_track_details, df_media, on='media_key', how='left')
df_track_details.head()

,product_key,Name,album_key,media_key,genre_key,Composer,Milliseconds,Bytes,UnitPrice,artist,album,genre,media
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,AC/DC,For Those About To Rock We Salute You,Rock,MPEG audio file
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99,Accept,Balls to the Wall,Rock,Protected AAC audio file
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99,Accept,Restless and Wild,Rock,Protected AAC audio file
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99,Accept,Restless and Wild,Rock,Protected AAC audio file
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99,Accept,Restless and Wild,Rock,Protected AAC audio file


In [207]:
#drop excess keys no longer needed since all in one dataframe now!
df_track_details.drop(['album_key', 'media_key','genre_key'], axis=1, inplace=True)
df_track_details.head()

,product_key,Name,Composer,Milliseconds,Bytes,UnitPrice,artist,album,genre,media
0,1,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,AC/DC,For Those About To Rock We Salute You,Rock,MPEG audio file
1,2,Balls to the Wall,None,342562,5510424,0.99,Accept,Balls to the Wall,Rock,Protected AAC audio file
2,3,Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99,Accept,Restless and Wild,Rock,Protected AAC audio file
3,4,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99,Accept,Restless and Wild,Rock,Protected AAC audio file
4,5,Princess of the Dawn,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99,Accept,Restless and Wild,Rock,Protected AAC audio file


In [208]:
dataframe = df_track_details
table_name = 'dim_product'
primary_key = 'product_key'
db_operation = "insert"

try:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)
except:
    print("Error: unable to send data")